In [1]:
import numpy as np
import pandas as pd

import numpy as np
import pandas as pd
import datetime as dt 
import sqlite3

path='../../../foothill/raw/'
date_list=['2021-04-12','2021-04-13','2021-04-14','2021-04-15','2021-04-16']

meta=pd.read_csv('../../meta/Foothill_detectors.csv')


In [2]:
manual=pd.DataFrame([])
for d in date_list:

    occupancy_of_chunk_150_lenght=pd.read_csv('../../occupancy/300/'+d+'/'+'occupancy_of_chunk_150_lenght'+d+'.csv', index_col=[0])


    occupancy_of_chunk_150_lenght=occupancy_of_chunk_150_lenght.sort_values(['signal','detector_parameter','unique_cycle_id','chunk'])
    occupancy_of_chunk_150_lenght['below_second']=occupancy_of_chunk_150_lenght.groupby(['signal','detector_parameter','unique_cycle_id'])['seconds'].shift(-1)

    occupancy_of_chunk_150_lenght['point_difference']=occupancy_of_chunk_150_lenght['below_second']-occupancy_of_chunk_150_lenght['seconds']
    occupancy_of_chunk_150_lenght.start_cycle=pd.to_datetime(occupancy_of_chunk_150_lenght.start_cycle)
    occupancy_of_chunk_150_lenght['date']=occupancy_of_chunk_150_lenght.start_cycle.dt.date


    temp=occupancy_of_chunk_150_lenght[['signal','date','detector_parameter','unique_cycle_id','chunk','point_difference']]
    temp=temp[temp.point_difference<0]

    temp=temp.sort_values(['signal','date','detector_parameter','unique_cycle_id','chunk']).assign(
    first_ocup_drop_chunk = 
    temp
    .groupby(['signal','date','detector_parameter','unique_cycle_id'])
    .chunk
    .transform('min')
    )

    temp=temp.assign(
    max_occup_drop_value = 
    temp
    .groupby(['signal','date','detector_parameter','unique_cycle_id'])
    .point_difference
    .transform('min')
    )


    temp['drop_point']=np.where(temp.max_occup_drop_value==temp.point_difference, 'maximum_drop_point','na' )


    #alternative_solution_phase=pd.read_csv('../../phase_prob/'+d+'/'+'adjustment_left_merged_1'+d+'.csv', index_col=[0])
    alternative_solution_phase=pd.read_csv('../../phase_prob/300/'+d+'/'+'alternative_solution_phase_probability'+d+'.csv', index_col=[0])

    alternative_solution_phase.cycle_start=pd.to_datetime(alternative_solution_phase.cycle_start)
    alternative_solution_phase['date']=alternative_solution_phase.cycle_start.dt.date


    temp2=alternative_solution_phase[(alternative_solution_phase['event_color']=='Green') & (alternative_solution_phase['location']=='first') ].sort_values(['signal','parameter','unique_cycle_id','b']).groupby(['signal','parameter','unique_cycle_id']).first().reset_index()
    temp2=temp2[['signal', 'date','parameter', 'cycle_id', 'unique_cycle_id', 'event_color',
       'location', 'b',  'below_chunk', 'event_start_time',
       'event_end_time', 'cycle_start', 'cycle_end', 'event', 'chunk_start',
       'chunk_end']]


    combination=temp2.merge(temp, how='left', on=['signal','unique_cycle_id','date'])[['signal','date','parameter','detector_parameter','cycle_id','unique_cycle_id','chunk','b','first_ocup_drop_chunk','max_occup_drop_value','drop_point']]
    combination['chunk_difference']=combination['chunk']-combination['b']
    combination=combination[( combination['chunk_difference']<=2) & ( combination['chunk_difference']>=0) ].rename(columns={'chunk':'occupancy_drop_chunk','b':'phase_first_green_chunk'})
    combination=combination.reset_index(drop=True)
    combination['drop_id']=combination.index

    combination=combination.merge(meta, left_on=['signal','detector_parameter'], right_on=['SignalID','DetChannel'], how='left')
    method1=combination.groupby(['signal','date','detector_parameter','Phase','parameter','chunk_difference'])['drop_id'].count().reset_index()
    method1=method1.rename(columns={'drop_id':'count_of_drop'})

    method2=method1.assign(
        max_count_drop = 
        method1
        .groupby(['signal','date','detector_parameter','Phase','chunk_difference'])
        .count_of_drop
        .transform('max')
        )

    method3=method2[method2['max_count_drop']==method2['count_of_drop']].sort_values(['signal','date','detector_parameter','Phase','chunk_difference','max_count_drop'])

    method4=method3.assign(
    frequent_parameter_count= 
    method3
    .groupby(['signal','date','detector_parameter','parameter'])
    .chunk_difference
    .transform('count')
    )



    #method4=method3.groupby(['signal','detector_parameter','parameter'])['chunk_difference'].count().reset_index()
    method5=method4.drop_duplicates(['signal','date','detector_parameter','Phase','parameter']).sort_values(['signal','detector_parameter','Phase','parameter'])

    method6=method5[['signal','date','detector_parameter','Phase','parameter','frequent_parameter_count']].assign(
    max_of_frequent_parameter_count= 
    method5
    .groupby(['signal','date','detector_parameter','Phase'])
    .frequent_parameter_count
    .transform('max')
    )

    method7=method6[method6['frequent_parameter_count']==method6['max_of_frequent_parameter_count']]
    manual=manual.append(method7)

In [16]:
manual.signal.unique()

array([7216, 7217, 7218, 7219, 7220, 7221, 7222, 7223, 7274, 7371, 7503],
      dtype=int64)

In [3]:
manual['correct']='na'
manual['correct']=np.where(manual['Phase']==manual['parameter'],1,0)

In [4]:
manual.to_csv('manual.csv')

In [5]:
summary=manual.groupby(['signal','detector_parameter','parameter','Phase'])['correct'].count().reset_index().rename(columns={'correct':'count'})
summary.sort_values(['signal','detector_parameter','parameter','count'])

,signal,detector_parameter,parameter,Phase,count
0,7216,10,2,2.0,1
1,7216,10,6,2.0,4
2,7216,12,1,6.0,2
3,7216,12,2,6.0,1
4,7216,12,6,6.0,3
...,...,...,...,...,...
294,7371,51,8,8.0,5
295,7503,2,4,2.0,5
296,7503,4,4,6.0,5
297,7503,5,4,4.0,5


In [23]:
summary1=summary[summary['Phase']<9]
summary1=summary1.merge(meta, left_on=['signal','detector_parameter'], right_on=['SignalID','DetChannel'])
summary1=summary1[summary1['DetectionTypes']!='Advanced Count']
summary1.groupby(['signal'])['detector_parameter'].nunique().reset_index()

,signal,detector_parameter
0,7216,28
1,7217,24
2,7218,24
3,7219,30
4,7220,20
5,7221,5
6,7223,2
7,7371,16
8,7503,2


In [17]:
summary.signal.unique()

array([7216, 7217, 7218, 7219, 7220, 7221, 7222, 7223, 7274, 7371, 7503],
      dtype=int64)

In [19]:
summary.loc[summary.groupby(['signal','detector_parameter'])['count'].idxmax()]

,signal,detector_parameter,parameter,Phase,count
1,7216,10,6,2.0,4
4,7216,12,6,6.0,3
6,7216,17,5,5.0,5
7,7216,18,2,2.0,5
8,7216,19,2,2.0,5
...,...,...,...,...,...
293,7371,51,4,8.0,5
295,7503,2,4,2.0,5
296,7503,4,4,6.0,5
297,7503,5,4,4.0,5


In [6]:
result=summary.loc[summary.groupby(['signal','detector_parameter'])['count'].idxmax()]
result['correct']=np.where(result['parameter']==result['Phase'],1,0)
result

,signal,detector_parameter,parameter,Phase,count,correct
1,7216,10,6,2.0,4,0
4,7216,12,6,6.0,3,1
6,7216,17,5,5.0,5,1
7,7216,18,2,2.0,5,1
8,7216,19,2,2.0,5,1
...,...,...,...,...,...,...
293,7371,51,4,8.0,5,0
295,7503,2,4,2.0,5,0
296,7503,4,4,6.0,5,0
297,7503,5,4,4.0,5,1


In [18]:
result.signal.unique()

array([7216, 7217, 7218, 7219, 7220, 7221, 7223, 7371, 7503], dtype=int64)

In [7]:
meta

,SignalID,DetChannel,Direction,Phase,DetectionTypes,MovementType,LaneNumber,AdvShift
0,7216,29,EB,1,Stop Bar Presence,Left,1,0.000000
1,7216,30,EB,1,Stop Bar Presence,Left,2,0.000000
2,7216,34,EB,1,Lane-by-lane Count,Left,1,0.000000
3,7216,35,EB,1,Lane-by-lane Count,Left,2,0.000000
4,7216,10,WB,2,Advanced Count,Thru,1,5.795323
...,...,...,...,...,...,...,...,...
180,7371,51,WB,8,Lane-by-lane Count,Thru-Left,1,0.000000
181,7503,2,NB,2,Advanced Count,Thru,1,6.818027
182,7503,4,SB,6,Advanced Count,Thru,1,6.818027
183,7503,5,EB,4,Stop Bar Presence,Left,1,0.000000


In [8]:
result=result[result['Phase']<9]
result=result.merge(meta, left_on=['signal','detector_parameter'], right_on=['SignalID','DetChannel'])
result=result[result['DetectionTypes']!='Advanced Count']

In [9]:
result.to_csv('result.csv')

In [20]:
result[result['DetectionTypes'].isin(['Stop Bar Presence','Lane-by-lane Count'])].groupby(['signal'])['detector_parameter'].nunique().reset_index()

,signal,detector_parameter
0,7216,28
1,7217,24
2,7218,24
3,7219,30
4,7220,20
5,7221,5
6,7223,2
7,7371,16
8,7503,2


In [32]:
result.groupby(['signal','DetectionTypes'])['detector_parameter'].count().reset_index()

,signal,DetectionTypes,detector_parameter
0,7216,Lane-by-lane Count,14
1,7216,Stop Bar Presence,14
2,7217,Lane-by-lane Count,12
3,7217,Stop Bar Presence,12
4,7218,Lane-by-lane Count,12
5,7218,Stop Bar Presence,12
6,7219,Lane-by-lane Count,15
7,7219,Stop Bar Presence,15
8,7220,Lane-by-lane Count,10
9,7220,Stop Bar Presence,10


In [30]:
result.groupby(['signal','DetectionTypes','correct'])['detector_parameter'].count().reset_index()

,signal,DetectionTypes,correct,detector_parameter
0,7216,Lane-by-lane Count,0,2
1,7216,Lane-by-lane Count,1,12
2,7216,Stop Bar Presence,0,2
3,7216,Stop Bar Presence,1,12
4,7217,Lane-by-lane Count,0,5
5,7217,Lane-by-lane Count,1,7
6,7217,Stop Bar Presence,0,4
7,7217,Stop Bar Presence,1,8
8,7218,Lane-by-lane Count,0,7
9,7218,Lane-by-lane Count,1,5


In [31]:
result.groupby(['DetectionTypes','correct'])['detector_parameter'].count().reset_index()

,DetectionTypes,correct,detector_parameter
0,Lane-by-lane Count,0,26
1,Lane-by-lane Count,1,48
2,Stop Bar Presence,0,21
3,Stop Bar Presence,1,56


In [33]:
g=result.groupby('signal')
detailed_accuracy=pd.DataFrame([])
for sets, data in g:
    accuracy=len(data[data['correct']==1])/len(data)
    df=pd.DataFrame({'signal':sets, 'accuracy':accuracy,'number of detectors':len(data)}, index=[0])
    detailed_accuracy=detailed_accuracy.append(df)
detailed_accuracy

,signal,accuracy,number of detectors
0,7216,0.857143,28
0,7217,0.625000,24
0,7218,0.583333,24
0,7219,0.700000,30
0,7220,0.800000,20
0,7221,1.000000,5
0,7223,0.500000,2
0,7371,0.375000,16
0,7503,1.000000,2


In [35]:
g=result.groupby(['signal','DetectionTypes'])
detailed_accuracy=pd.DataFrame([])
for sets, data in g:
    accuracy=len(data[data['correct']==1])/len(data)
    df=pd.DataFrame({'signal':sets[0],'Type':sets[1], 'accuracy':accuracy,'number of detectors':len(data)}, index=[0])
    detailed_accuracy=detailed_accuracy.append(df)
detailed_accuracy

,signal,Type,accuracy,number of detectors
0,7216,Lane-by-lane Count,0.857143,14
0,7216,Stop Bar Presence,0.857143,14
0,7217,Lane-by-lane Count,0.583333,12
0,7217,Stop Bar Presence,0.666667,12
0,7218,Lane-by-lane Count,0.416667,12
0,7218,Stop Bar Presence,0.750000,12
0,7219,Lane-by-lane Count,0.733333,15
0,7219,Stop Bar Presence,0.666667,15
0,7220,Lane-by-lane Count,0.700000,10
0,7220,Stop Bar Presence,0.900000,10


# after removing the low data and no signal detectors 


In [37]:
result_filtered=pd.read_csv('result_filtered.csv')

In [38]:
# after removing the low data and no signal detectors 
g=result_filtered.groupby('signal')
detailed_accuracy=pd.DataFrame([])
for sets, data in g:
    accuracy=len(data[data['correct']==1])/len(data)
    df=pd.DataFrame({'signal':sets, 'accuracy':accuracy,'number of detectors':len(data)}, index=[0])
    detailed_accuracy=detailed_accuracy.append(df)
detailed_accuracy

,signal,accuracy,number of detectors
0,7216,1.000000,24
0,7217,0.789474,19
0,7218,0.875000,16
0,7219,0.840000,25
0,7220,0.941176,17
0,7221,1.000000,5
0,7223,0.500000,2
0,7371,0.666667,9
0,7503,1.000000,2
